In [1]:
from fastai.vision.all import *
from fastbook import *

load images

In [2]:
path = untar_data(URLs.MNIST_SAMPLE)

threes_train = (path/'train'/'3').ls().sorted()
sevens_train = (path/'train'/'7').ls().sorted()

threes_valid = (path/'valid'/'3').ls().sorted()
sevens_valid = (path/'valid'/'7').ls().sorted()

In [3]:
seven_tensors = [tensor(Image.open(o)) for o in sevens_train]
three_tensors = [tensor(Image.open(o)) for o in threes_train]
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255

In [4]:
valid_3_tens = torch.stack([tensor(Image.open(o)) for o in threes_valid])
valid_7_tens = torch.stack([tensor(Image.open(o)) for o in sevens_valid])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = valid_7_tens.float()/255

Dataset

In [5]:
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
train_y = tensor([1]*len(threes_train) + [0]*len(sevens_train)).unsqueeze(1)

dset = list(zip(train_x,train_y))

In [6]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)

valid_dset = list(zip(valid_x,valid_y))

In [7]:
dl = DataLoader(dset, batch_size=256)
valid_dl = DataLoader(valid_dset, batch_size=256)

dls = DataLoaders(dl, valid_dl)

loss and accuracy

In [8]:
# is_3?-model
def mnist_loss(predictions, targets):
    return torch.where(targets==1, 1-predictions, predictions).mean()

def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

Learner

In [9]:
# linear model
linear_model = nn.Linear(28*28,1)
w,b = linear_model.parameters()
w.shape,b.shape

(torch.Size([1, 784]), torch.Size([1]))

In [10]:
learn = Learner(dls, linear_model, opt_func=SGD, 
               loss_func=mnist_loss, metrics=batch_accuracy)

In [11]:
#train model
learn.fit(10, lr=1)

epoch,train_loss,valid_loss,batch_accuracy,time
0,-82.954857,-335.733124,0.953386,00:00
1,-309.328003,-672.050049,0.953386,00:00
2,-570.715759,-1008.366821,0.953386,00:00
3,-854.969299,-1344.683716,0.953386,00:00
4,-1152.399658,-1681.000610,0.953386,00:00
5,-1456.732910,-2017.317505,0.953386,00:00
6,-1764.437622,-2353.634033,0.953386,00:00
7,-2073.706299,-2689.951416,0.953386,00:00
8,-2383.674561,-3026.267822,0.953386,00:00
9,-2693.947021,-3362.584961,0.953386,00:00
